In [25]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 sumy pydantic==1.10.10

     ---------------------------------------- 0.0/150.0 kB ? eta -:--:--
     -- ------------------------------------- 10.2/150.0 kB ? eta -:--:--
     ----- ------------------------------- 20.5/150.0 kB 217.9 kB/s eta 0:00:01
     ------------ ------------------------ 51.2/150.0 kB 327.7 kB/s eta 0:00:01
     ---------------------------------- - 143.4/150.0 kB 853.3 kB/s eta 0:00:01
     ------------------------------------ 150.0/150.0 kB 747.4 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of crewai to determine which version is compatible with other requirements. This could take a while.

The conflict is caused by:
    The user requested pydantic==1.10.10
    crewai 0.28.8 depends on pydantic<3.0.0 and >=2.4.2

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict



ERROR: Cannot install crewai==0.28.8 and pydantic==1.10.10 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [94]:
pip install spider-client 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for spider-client: filename=spider_client-0.0.72-py3-none-any.whl size=12953 sha256=eaad44178520951c9875ae97f24451cef4ed3411e8a4a66f0eca3067f2f33b53
  Stored in directory: c:\users\sriya\appdata\local\pip\cache\wheels\d1\3f\9d\4f73e53fd1a852257b66e5b40c7dd94b160a983c7179139ecf
Successfully built spider-client
Note: you may need to restart the kernel to use updated packages.


In [42]:
!pip install vertexai

In [2]:
# Handling Warnings
import warnings
warnings.filterwarnings('ignore')

Import libraries, APIs and LLMs 

In [43]:
from crewai import Agent, Crew, Task
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, BaseTool,SeleniumScrapingTool
from typing import Type , List , Dict , Optional
from pydantic.v1 import BaseModel, Field , PrivateAttr
import json
import matplotlib.pyplot as plt
import statistics
import re
import requests
import vertexai

In [38]:
from dotenv import load_dotenv
import os
load_dotenv()
gemini_api_key = os.getenv("GEMINI_AI_API_KEY")
google_search_api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
google_search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")
serper_api_key = os.getenv("SERPER_API_KEY")
#open_ai_api_key = os.getenv("OPENAI_API_KEY")

Initialize the tools and creating few custom tools

In [76]:
#search_tool = SerperDevTool()
os.environ['GEMINI_AI_="google/gemini-pro")
scrape_tool = SeleniumScrapingTool()
search_tool = SerperDevTool(
    search_url="https://google.serper.dev/scholar",
    n_results=5,
    api_key = serper_api_key
)
#summarizer_tool = LsaSummarizer()

Data Analysis Tool

In [45]:
class DataAnalysisInput(BaseModel):
    data: str = Field(..., description="Structured data as a JSON string for analysis.")  # Required field

class DataAnalysisTool(BaseTool):
    name: str = "Data Analysis Tool"
    description: str = (
        "Analyzes structured data to provide basic insights such as entry count, mean, median, "
        "and other relevant statistics."
    )
    args_schema: Type[BaseModel] = DataAnalysisInput

    def _run(self, data: str) -> str:
        try:
            structured_data = json.loads(data)
        except json.JSONDecodeError:
            return "Invalid data format. Please provide data in JSON format."

        # Check if the structured data is a list of numbers
        if isinstance(structured_data, list) and all(isinstance(i, (int, float)) for i in structured_data):
            entry_count = len(structured_data)
            mean_value = statistics.mean(structured_data) if entry_count > 0 else None
            median_value = statistics.median(structured_data) if entry_count > 0 else None
            stdev_value = statistics.stdev(structured_data) if entry_count > 1 else None

            result = {
                "entry_count": entry_count,
                "mean": mean_value,
                "median": median_value,
                "standard_deviation": stdev_value
            }
            return json.dumps(result)

        return "Unsupported data format or structure. Please provide a list of numeric values."

data_analysis_tool = DataAnalysisTool()


Text Summarization Tool

In [46]:
class SummarizationInput(BaseModel):
    """Input schema for TextSummarizationTool."""
    text: str = Field(..., description="The text to be summarized.")  # Required field

class TextSummarizationTool(BaseTool):
    name: str = "Text Summarization Tool"
    description: str = "Summarizes text into concise key points or a shorter version."
    args_schema: Type[BaseModel] = SummarizationInput

    def _run(self, text: str) -> str:
        openai.api_key = os.getenv("OPENAI_API_KEY")
        try:
            response = openai.Completion.create(
                engine="text-davinci-003",
                prompt=f"Summarize the following text:\n{text}",
                max_tokens=100,  # Adjust based on desired summary length
                temperature=0.3,
            )
            return response.choices[0].text.strip() if response.choices else "No summary generated."
        except Exception as e:
            return f"Error during summarization: {e}"

text_summarization_tool = TextSummarizationTool()

Visualization Tool

In [47]:
class VisualizationInput(BaseModel):
    """Input schema for VisualizationTool."""
    data: str = Field(..., description="Data to visualize in JSON format.")
    chart_type: str = Field(..., description="Type of chart to create (e.g., 'bar', 'line').")

class VisualizationTool(BaseTool):
    name: str = "Visualization Tool"
    description: str = "Provides visual representations of data trends and proposed use cases."
    args_schema: Type[BaseModel] = VisualizationInput

    def _run(self, data: str, chart_type: str) -> str:
        try:
            # Parse data
            parsed_data = json.loads(data)

            # Generate visualizations based on chart_type
            if chart_type == 'bar':
                plt.bar(parsed_data['labels'], parsed_data['values'])
                plt.title('Bar Chart')
                plt.xlabel('Labels')
                plt.ylabel('Values')
            elif chart_type == 'line':
                plt.plot(parsed_data['labels'], parsed_data['values'])
                plt.title('Line Chart')
                plt.xlabel('Labels')
                plt.ylabel('Values')
            else:
                return "Unsupported chart type. Please choose 'bar' or 'line'."

            # Save the plot as an image file
            plt.savefig("visualization.png")
            plt.close()  # Close the plot to free memory
            return "Visualization created and saved as 'visualization.png'."
        
        except json.JSONDecodeError:
            return "Invalid data format. Please provide data in JSON format."
        except Exception as e:
            return f"Error during visualization: {e}"

In [48]:
visualization_tool = VisualizationTool()

Bookmark Tool

In [49]:
class BookmarkInput(BaseModel):
    """Input schema for Bookmark Tool."""
    name: str = Field(..., description="A name for the bookmarked resource.")
    url: str = Field(..., description="The URL of the dataset or resource to bookmark.")
    description: str = Field(None, description="A brief description of the bookmarked resource.")

class RetrieveBookmarksInput(BaseModel):
    """Input schema for retrieving bookmarks."""
    keyword: str = Field(None, description="Keyword to search within bookmark names and descriptions for easy retrieval.")

class BookmarkTool(BaseTool):
    name: str = "Bookmark Tool"
    description: str = "Organizes and saves frequently accessed datasets and resources for easy retrieval and sharing with stakeholders."
    args_schema: Type[BaseModel] = BookmarkInput
    
    # Define bookmarks as a Pydantic field with a default empty list
    bookmarks: List[dict] = Field(default_factory=list, description="List of bookmarked resources.")
    
    def _run(self, name: str, url: str, description: str = "") -> str:
        # Add a new bookmark
        bookmark = {"name": name, "url": url, "description": description}
        self.bookmarks.append(bookmark)
        return f"Bookmark '{name}' has been added successfully."
    
    def retrieve_bookmarks(self, keyword: str = None) -> str:
        # Retrieve bookmarks, filtered by keyword if provided
        if keyword:
            filtered_bookmarks = [b for b in self.bookmarks if keyword.lower() in (b['name'].lower() + b['description'].lower())]
            return json.dumps(filtered_bookmarks, indent=2)
        else:
            return json.dumps(self.bookmarks, indent=2)

In [50]:
bookmark_tool = BookmarkTool()

Documents Assembly Tool

In [51]:
class DocumentAssemblyInput(BaseModel):
    use_cases: List[Dict[str, str]] = Field(..., description="List of use cases with title and description.")
    references: List[Dict[str, str]] = Field(..., description="List of references with title and URL.")
    format: str = Field("md", description="Format of the document to generate ('txt' or 'md').")

class DocumentAssemblyTool(BaseTool):
    name: str = "Document Assembly Tool"
    description: str = "Combines insights, suggestions, and resources into a structured document."
    args_schema: Type[BaseModel] = DocumentAssemblyInput

    def _run(self, use_cases: List[Dict[str, str]], references: List[Dict[str, str]], format: str) -> str:
        # Validate format
        if format not in ['txt', 'md']:
            return "Invalid format specified. Please choose 'txt' or 'md'."

        # Construct the document content
        document_content = ""
        document_content += "# Proposal Document\n\n"

        # Add use cases to the document
        document_content += "## Use Cases\n"
        for use_case in use_cases:
            document_content += f"### {use_case['title']}\n"
            document_content += f"{use_case['description']}\n\n"

        # Add references to the document
        document_content += "## References\n"
        for reference in references:
            if format == 'md':
                document_content += f"- [{reference['title']}]({reference['url']})\n"
            else:
                document_content += f"- {reference['title']}: {reference['url']}\n"

        # Save the document
        file_extension = 'md' if format == 'md' else 'txt'
        file_name = f"proposal_document.{file_extension}"
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(document_content)

        return f"Document '{file_name}' created successfully."


In [52]:
document_assembly_tool = DocumentAssemblyTool()

Reference Checker Tool

In [53]:
class ReferenceCheckerInput(BaseModel):
    references: List[Dict[str, str]] = Field(..., description="List of references with title and URL.")

class ReferenceCheckerTool(BaseTool):
    name: str = "Reference Checker Tool"
    description: str = "Ensures all referenced sources are correctly cited and formatted within the proposal document."
    args_schema: Type[BaseModel] = ReferenceCheckerInput

    def _run(self, references: List[Dict[str, str]]) -> str:
        formatted_references = []

        for reference in references:
            title = reference.get('title', '').strip()
            url = reference.get('url', '').strip()

            # Basic validation for title and URL
            if not title or not url:
                return "Error: Each reference must have a title and a URL."

            if not self._is_valid_url(url):
                return f"Error: Invalid URL format for reference '{title}'."

            # Format the reference
            formatted_references.append(f"- [{title}]({url})")

        return "Formatted References:\n" + "\n".join(formatted_references)

    def _is_valid_url(self, url: str) -> bool:
        # Simple regex for validating a URL
        regex = re.compile(
            r'^(?:http|ftp)s?://'  # http:// or https://
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'  # domain...
            r'localhost|'  # localhost...
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}|'  # IPv4
            r'\[?[A-F0-9]*:[A-F0-9:]+\]?)'  # IPv6
            r'(?::\d+)?'  # optional port
            r'(?:/?|[/?]\S+)$', re.IGNORECASE)
        return re.match(regex, url) is not None


In [54]:
reference_checker_tool = ReferenceCheckerTool()

Link Formatter Tool

In [55]:
class LinkFormatterInput(BaseModel):
    text: str = Field(..., description="Text containing raw URLs to be formatted as clickable Markdown links.")

class LinkFormatterTool(BaseTool):
    name: str = "Link Formatter Tool"
    description: str = "Identifies raw URLs in the text and formats them as clickable Markdown links."
    args_schema: Type[BaseModel] = LinkFormatterInput

    def _run(self, text: str) -> str:
        # Regex pattern to find URLs
        url_pattern = r'(https?://[^\s]+)'

        # Function to convert URLs to Markdown format [URL](URL)
        def format_link(match):
            url = match.group(0)
            return f"[{url}]({url})"

        # Substitute raw URLs with formatted Markdown links
        formatted_text = re.sub(url_pattern, format_link, text)

        return formatted_text

In [56]:
link_formatter_tool = LinkFormatterTool()

Feedback and Interation Tool

In [57]:
class FeedbackEntry(BaseModel):
    feedback_id: str = Field(..., description="Unique identifier for each feedback entry.")
    stakeholder_name: str = Field(..., description="Name of the stakeholder providing feedback.")
    comments: str = Field(..., description="Feedback comments for the draft proposal.")
    suggestions: Optional[str] = Field(None, description="Suggestions for improvement.")

class FeedbackAndIterationInput(BaseModel):
    proposal_id: str = Field(..., description="ID of the proposal document to gather feedback on.")
    feedback_entries: List[FeedbackEntry] = Field(..., description="List of feedback entries provided by stakeholders.")

class FeedbackAndIterationTool(BaseTool):
    name: str = "Feedback and Iteration Tool"
    description: str = (
        "Collects feedback from stakeholders on draft proposals and refines them iteratively. "
        "Each feedback entry is stored with stakeholder details and comments, allowing the agent "
        "to update the proposal document based on received input."
    )
    args_schema: Type[BaseModel] = FeedbackAndIterationInput

    # Initialize feedback storage (could be a database in production)
    feedback_storage: dict = {}

    def _run(self, proposal_id: str, feedback_entries: List[FeedbackEntry]) -> str:
        # Check if feedback exists for this proposal, if not initialize
        if proposal_id not in self.feedback_storage:
            self.feedback_storage[proposal_id] = []
        
        # Add feedback entries to storage
        for feedback in feedback_entries:
            self.feedback_storage[proposal_id].append(feedback.dict())
        
        # Summarize feedback process
        feedback_count = len(self.feedback_storage[proposal_id])
        return f"Feedback successfully collected for proposal {proposal_id}. Total feedback entries: {feedback_count}."

    def refine_proposal(self, proposal_text: str, proposal_id: str) -> str:
        """Refines the proposal based on feedback entries."""
        if proposal_id not in self.feedback_storage or not self.feedback_storage[proposal_id]:
            return "No feedback available for refinement."

        # Iterate over feedback entries for this proposal
        refined_proposal = proposal_text
        refinement_log = []
        
        for entry in self.feedback_storage[proposal_id]:
            comments = entry['comments']
            suggestions = entry.get('suggestions')
            
            # Simple refinement by appending comments and suggestions for this demo
            refined_proposal += f"\n\n# Feedback from {entry['stakeholder_name']}\n"
            refined_proposal += f"- Comments: {comments}\n"
            if suggestions:
                refined_proposal += f"- Suggestions: {suggestions}\n"
            
            # Log each refinement step for traceability
            refinement_log.append(f"Refined based on feedback from {entry['stakeholder_name']}")
        
        return refined_proposal + "\n\n" + "\n".join(refinement_log)

In [58]:
feedback_iteration_tool = FeedbackAndIterationTool()

Content Validator Tool

In [59]:
class ContentValidatorInput(BaseModel):
    document_text: str = Field(..., description="Text of the proposal document to validate.")
    required_sections: List[str] = Field(..., description="List of required sections to check for completeness.")

class ContentValidatorTool(BaseTool):
    name: str = "Content Validator Tool"
    description: str = (
        "Validates the completeness and accuracy of a compiled proposal. "
        "Checks for missing sections, broken links, and incomplete references to ensure document quality."
    )
    args_schema: Type[BaseModel] = ContentValidatorInput

    def _run(self, document_text: str, required_sections: List[str]) -> str:
        # Track validation results
        validation_report = []

        # Check for required sections
        missing_sections = [section for section in required_sections if section not in document_text]
        if missing_sections:
            validation_report.append(f"Missing sections: {', '.join(missing_sections)}.")
        else:
            validation_report.append("All required sections are present.")

        # Check for broken links
        validation_report.append(self._check_links(document_text))

        # Check for incomplete references
        validation_report.append(self._check_references(document_text))

        return "\n".join(validation_report)

    def _check_links(self, text: str) -> str:
        """Identify and validate URLs in the document."""
        url_pattern = r'(https?://[^\s\)]+)'
        urls = re.findall(url_pattern, text)
        broken_links = []

        for url in urls:
            try:
                response = requests.head(url, allow_redirects=True, timeout=5)
                if response.status_code >= 400:
                    broken_links.append(url)
            except requests.RequestException:
                broken_links.append(url)

        if broken_links:
            return f"Broken links detected: {', '.join(broken_links)}."
        else:
            return "No broken links detected."

    def _check_references(self, text: str) -> str:
        """Detect placeholders or missing references in text."""
        incomplete_ref_pattern = r'\[.*?TODO.*?\]'
        incomplete_refs = re.findall(incomplete_ref_pattern, text)

        if incomplete_refs:
            return f"Incomplete references detected: {', '.join(incomplete_refs)}."
        else:
            return "No incomplete references detected."

In [60]:
content_validator_tool = ContentValidatorTool()

In [61]:
print(type(search_tool))  # Should print: <class 'your_module.MyTool'>
print(type(scrape_tool))  # Should print: <class 'crewai_tools.ScrapeWebsiteTool'>
print(type(text_summarization_tool))  # Should print: <class 'crewai_tools.TextSummarizationTool'>
print(type(data_analysis_tool))  

<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevTool'>
<class 'crewai_tools.tools.selenium_scraping_tool.selenium_scraping_tool.SeleniumScrapingTool'>
<class '__main__.TextSummarizationTool'>
<class '__main__.DataAnalysisTool'>


**Creating Agents**

1. Research Agent
2. Market Analyst Agent
3. Use Case Generation Agent
4. Resource Collecton Agent
5. Proposal Finalist Agent

Research Agent

In [62]:
research_agent = Agent(
    role = "Research Agent",
    goal = "Conduct industry and company-specific research to identify the company's market segment, key offerings, and strategic focus areas",
    backstory = ("Uses browser tool to search for information about {company_name} "
            "in the {industry} sector. It focuses on understanding the industry, segmenting the market, "
            "and identifying the company's key offerings and strategic focus areas (e.g., operations, supply chain, customer experience)."),
    verbose = True,
    allow_delegation=True,
)

Market Analyst Agent

In [63]:
market_analyst_agent = Agent(
    role = "Market Analyst Agent",
    goal = "To identify key industry trends, standards, and emerging AI/ML opportunities within the target industry, providing insights that help the company leverage AI/ML to optimize operations, improve customer satisfaction, and drive efficiency.",
    backstory =("conducting in-depth analysis of the target industry’s market standards and trends, focusing on AI and ML advancements"
                " This agent explores recent developments, key innovations, and prominent use cases of AI/ML within the industry"
                " It identifies opportunities where AI, ML, or Generative AI can add value to processes or enhance customer experience"
                " By evaluating best practices, the Market Analyst Agent aims to recommend industry-relevant AI solutions that align with the company’s strategic objectives, improving both operational efficiency and customer engagement."),
)    

Use Case Generation Agent

In [64]:
usecase_generation_agent = Agent(
    role = "Use Case Generation Agent",
    goal = "To analyze industry data and generate actionable use cases that leverage AI/ML technologies, providing organizations with insights into how to implement predictive maintenance, personalized customer support, and other innovative solutions that enhance operational efficiency and customer experience.",
    backstory = ("The Use Case Generation Agent was created to bridge the gap between complex industry data and practical applications of AI/ML technologies. "
    "With a deep understanding of various industries, including manufacturing and retail, this agent meticulously examines historical and current data trends, "
    "identifies potential areas for improvement, and crafts tailored use cases. By employing GPT-based models and rule-based reasoning, the agent generates insightful recommendations, "
    "empowering organizations to harness the full potential of AI/ML for optimizing operations and enhancing customer engagement. "
    "Whether it's predicting equipment failures in manufacturing or crafting personalized customer interactions in retail, "
    "this agent is equipped to deliver valuable use cases that drive innovation."),
)

Resource Collector Agent

In [65]:
resource_collector_agent = Agent(
    role = "Resource Collection Agent",
    goal = "To efficiently gather high-quality datasets and reference materials from sources like Kaggle, GitHub, and Hugging Face, enabling data scientists, analysts, and developers to accelerate their work on machine learning and AI projects.",
    backstory =  ("The Resource Collection Agent was developed to streamline the often time-consuming process of locating "
    "relevant datasets and resources for data-driven projects. With access to platforms like Kaggle, GitHub, "
    "and Hugging Face, this agent specializes in searching and curating datasets and reference materials based "
    "on specific project needs, such as image datasets for computer vision or text corpora for NLP. By leveraging "
    "its understanding of each platform’s structure and filtering capabilities, the agent ensures that users receive "
    "accurate, curated collections of resources, significantly reducing the time needed for project preparation. "
    "The agent continually updates its knowledge to track newly released datasets and materials, making it a reliable "
    "guide to the ever-growing world of open-source data and machine learning resources."),
)

Proposal Finalist Agent

In [66]:
proposal_finalist_agent = Agent(
    role = "Proposal Finalist Agent",
    goal = "Aims to create a cohesive, well-formatted proposal document that integrates suggested use cases, provides accessible resources, and includes a clear list of references to enhance credibility and facilitate seamless sharing with stakeholders.",
    backstory = ("The Proposal Finalist Agent was developed to assist teams in turning insights and research findings into structured, actionable proposals. "
    "It organizes extensive data, resources, and references, ensuring clear presentation with clickable links and credible citations. "
    "This agent compiles essential details into an accessible format, helping teams progress from insights to actionable strategies, "
    "and providing value to both technical and non-technical stakeholders."),
)

**Creating Tasks**

In [67]:
research = Task(
    description=(
        "Conduct an in-depth analysis of {company_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=("A comprehensive report on {company_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."),
    tools = [search_tool, scrape_tool],
    #openai_api_key = os.getenv("OPENAI_API_KEY"),
    agent = research_agent
)

In [68]:
market_analysis_task = Task(
    description = ("Analyze industry-specific trends, benchmarks, and advancements in AI and machine learning. Identify emerging opportunities to drive innovation and competitive advantage within the target sector. Provide actionable insights and recommend areas for strategic AI/ML investments and development."),
    expected_output = ("Develop a comprehensive market analysis report to uncover key trends, growth opportunities, and potential market gaps. This report should include data-driven insights, visualizations, and actionable recommendations to support strategic business decisions."),
    tools = [search_tool, scrape_tool, text_summarization_tool, data_analysis_tool],
    agent = market_analyst_agent,
)

In [69]:
use_case_generation_task = Task(
    description = ("Generate use cases by mapping company goals to potential AI/ML solutions."),
    expected_output = ("Detailed list of AI/ML use cases relevant to company goals"
        "Brief impact analysis and feasibility overview for each use case"
        "Outline of data requirements and success metrics per use case"),
    agent = usecase_generation_agent,
    tools = [search_tool , text_summarization_tool , data_analysis_tool , visualization_tool]
)

In [70]:
resource_collection_task = Task (
    description = ("Identify and curate high-quality datasets along with associated reference resources from top data platforms, including Kaggle, GitHub, and HuggingFace."),
    expected_output = ("A well-organized document or spreadsheet with dataset names, links, descriptions, and notes on their relevance to the project."),
    agent = resource_collector_agent,
    tools = [search_tool , scrape_tool, text_summarization_tool, bookmark_tool ],
)
    

In [71]:
proposal_finalization_task = Task(
    description = ("Consolidate all components of the final proposal into a single document."
"Ensure that all resources referenced within the proposal are presented in a readable format (e.g., properly formatted text, bullet points, etc.)"
"Include references for suggested use cases to support the proposal's recommendations."
"Verify that all resource asset links are functional and clickable, allowing easy access for readers."),
    expected_output = ("Final proposal document in .txt or .md format."),
    agent = proposal_finalist_agent,
    tools = [document_assembly_tool, reference_checker_tool, link_formatter_tool, text_summarization_tool, feedback_iteration_tool , content_validator_tool]
)

Creating Crew

In [72]:
crew = Crew(
    agents=[research_agent , market_analyst_agent , usecase_generation_agent , resource_collector_agent , proposal_finalist_agent ],
    tasks=[research, 
           market_analysis_task,
           use_case_generation_task,
           resource_collection_task,
           proposal_finalization_task],
	
    verbose=2,
	memory=True
)

2024-11-03 22:06:08,207 - 34456 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [73]:
inputs = {"company_name":"AI Planet",
          "lead_name":"AI Planet",
          "industry" : "Online Learning Platform"}

In [74]:
results = crew.kickoff(inputs = inputs)

 [DEBUG]: == Working Agent: Research Agent
 [INFO]: == Starting Task: Conduct an in-depth analysis of AI Planet, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}